# 习题5.5


<li><span><a href="#（1）给出距离判别规则并分类，使用回代法评估" data-toc-modified-id="（1）给出距离判别规则并分类，使用回代法评估-"><span class="toc-item-num">&nbsp;&nbsp;</span>（1）给出距离判别规则并分类，使用回代法评估</a></span></li><li><span><a href="#（2）基于正态性&amp;方差齐性假定和先验概率的贝叶斯判别法" data-toc-modified-id="（2）基于正态性&amp;方差齐性假定和先验概率的贝叶斯判别法-"><span class="toc-item-num">&nbsp;&nbsp;</span>（2）基于正态性&amp;方差齐性假定和先验概率的贝叶斯判别法</a></span></li><li><span><a href="#（3）最小ECM贝叶斯判别法" data-toc-modified-id="（3）最小ECM贝叶斯判别法-"><span class="toc-item-num">&nbsp;&nbsp;</span>（3）最小ECM贝叶斯判别法</a></span></li><li><span><a href="#（4）Fisher判别法" data-toc-modified-id="（4）Fisher判别法-"><span class="toc-item-num">&nbsp;&nbsp;</span>（4）Fisher判别法


### （1）给出距离判别规则并分类，使用回代法评估

已知，$\hat{{W}({x})}=\hat{a^{\prime}}(x-\bar{x})$，其中 $\overline{{x}}=\frac{1}{2}\left(\overline{x_1}+\overline{x_2}\right)$；$\hat{a}=S_p^{-1}\left(\overline{x_1}-\overline{x_2}\right)$；$S_p=\frac{\left(n_1-1\right) S_1+\left(n_2-1\right) S_2}{n_1+n_2-2}$。

则判别规则为$\begin{cases}x \in \pi_1 & , \text { 若 } \hat{W(x)} \geq 0 \\ x \in \pi_2 & , \text { 若 } \hat{W(x)}<0\end{cases}$

In [55]:
# 读入数据
data = read.table('.//AMSA_data//《应用多元统计分析》（第6版）文本数据（以逗号为间隔）//exec5.5.csv',header = TRUE,sep = ',')
head(data) #训练集
data1 = subset(data, g =='1')[,-3]
data2 = subset(data, g =='2')[,-3]
datat = read.table('.//AMSA_data//《应用多元统计分析》（第6版）文本数据（以逗号为间隔）//exec5.5a.csv',header = TRUE,sep = ',')
head(datat) # 测试集

,x1,x2,g
,<dbl>,<dbl>,<int>
1,-1.9,3.2,1
2,-6.9,10.4,1
3,5.2,2.0,1
4,5.0,2.5,1
5,7.3,0.0,1
6,6.8,12.7,1


,x1,x2
,<dbl>,<int>
1,0.6,3


In [35]:
# 编写距离判别法的函数

jvlipanbie = function(X1, X2, testX=NULL, var.equal=FALSE){
  # 输入变量处理：X1，X2是两个总体，testX是测试集，默认总体协方差矩阵不同

  if (is.null(testX) == TRUE) testX = rbind(X1,X2) # 如果没有给定测试集，则对数据进行回测

  if (is.vector(testX) == TRUE) 
    testX = t(as.matrix(testX)) # 调整训练集的格式
  else if (is.matrix(testX) != TRUE)
    testX = as.matrix(testX)

  if (is.matrix(X1) == FALSE)
    X1 = as.matrix(X1)
  if (is.matrix(X2) == FALSE)
    X2 = as.matrix(X2) # 保证输入的总体是矩阵

  nx = nrow(testX)    # 测试集的大小
  pred = matrix(rep(0,nx), nrow=1, byrow=TRUE, dimnames=list("pred",1:nx))  # 生成长度为nx的0向量，用以存储预测的标签
  mu1 = colMeans(X1)
  mu2 = colMeans(X2) # 均值向量

  if (var.equal == TRUE){   # 当两总体同方差
    S = cov(rbind(X1,X2)) # 计算协方差
    Wx = mahalanobis(testX, mu2, S) - mahalanobis(testX, mu1, S) # 负的判别函数W(x)：到2的距离减去到1的距离
  }
  else{   # 当两总体异方差
    S1 = cov(X1)
    S2 = cov(X2)
    Wx = mahalanobis(testX, mu2, S2) - mahalanobis(testX, mu1, S1)
  }  
  
  for (i in 1:nx){
    if (Wx[i] >= 0)
      pred[i] = 1
    else
      pred[i] = 2  # 所以这里只适用于二分类的任务
  }

  return(pred) # 返回预测值
}

jvlipanbie(data1,data2,datat,TRUE) # 使用距离判别法得到的结果是1

,1
pred,1


In [48]:
# 回代法

Z =  jvlipanbie(data1,data2,NULL,TRUE) # 回代预测所属类别
table1 = table(g = data$g,Z) # 判别情况表
print(table1) 
n21 = 2;n12 = 1
n1 = length(t(data1))/2
n2 = length(t(data2))/2
print(sprintf('P(2|1)：%f',n21/n1))
print(sprintf('P(1|2)：%f',n12/n2))

   Z
g   1 2
  1 8 2
  2 1 9
[1] "P(2|1)：0.100000"
[1] "P(1|2)：0.050000"


因此，根据距离判别法预测明天会下雨，并且使用回代法得到误判概率P(2|1)=0.1，P(1|2)=0.05。

### （2）基于正态性&方差齐性假定和先验概率的贝叶斯判别法

In [60]:
library(MASS)
ld1 = lda(g~x1+x2, prior = c(0.3,0.7), data)
newg = predict(ld1, datat) # 预测所属的类别的结果
newg

$class
[1] 2
Levels: 1 2

$posterior
          1         2
1 0.4062312 0.5937688

$x
         LD1
1 -0.6377856

根据给定先验概率的贝叶斯判别法，预测明天不会下雨，并且下雨和不下雨的后验概率是0.4和0.6。

### （3）最小ECM贝叶斯判别法

In [69]:
bayesminECM = function(X1, X2, testX=NULL, var.equal=FALSE, rate=1){
  # 输入变量处理：X1，X2是两个总体，testX是测试集，默认总体协方差矩阵不同
  # rate = (c12 p2) / (c21p1)

  if (is.null(testX) == TRUE) testX = rbind(X1,X2) # 如果没有给定测试集，则对数据进行回测

  if (is.vector(testX) == TRUE) 
    testX = t(as.matrix(testX)) # 调整训练集的格式
  else if (is.matrix(testX) != TRUE)
    testX = as.matrix(testX)

  if (is.matrix(X1) == FALSE)
    X1 = as.matrix(X1)
  if (is.matrix(X2) == FALSE)
    X2 = as.matrix(X2) # 保证输入的总体是矩阵

  nx = nrow(testX)    # 测试集的大小
  pred = matrix(rep(0,nx), nrow=1, byrow=TRUE, dimnames=list("pred",1:nx))  # 生成长度为nx的0向量，用以存储预测的标签
  mu1 = colMeans(X1)
  mu2 = colMeans(X2) # 均值向量
  
  if (var.equal == TRUE){ # 两总体同方差
    S = cov(rbind(X1,X2))
    Wx = 1/2*(mahalanobis(testX,mu2,S) - mahalanobis(testX,mu1,S)) # 到2的距离减去到1的距离
    beta = log(rate) 
  }
  else{ # 当两总体不同方差
    S1 = cov(X1)
    S2 = cov(X2)
    Wx = 1/2*(mahalanobis(testX, mu2, S2) - mahalanobis(testX, mu1, S1))
    beta = log(det(S1)/det(S2))+log(rate)
  }
  
  for (i in 1:nx){
    if (Wx[i] >= beta)
      pred[i] = 1
    else
      pred[i] = 2  # 只适用于二分类的任务
  }

  return(pred) # 返回预测值
}

c12 = 1  ;c21 = 3*c12  # 假设
p1 = 0.3 ;p2 = 0.7
rate = (c12*p2) / (c21*p1)
bayesminECM(data1,data2,datat,TRUE,rate) # 使用距离判别法得到的结果是1

,1
pred,1


在给定相对代价和先验概率之后，贝叶斯判别法的结果是1，说明第二天会下雨，不必为明天的活动准备。

### （4）Fisher判别法

In [67]:
n = dim(data)[1] 
k = 1 #2-1

library(MASS)
ld2 = lda(g~x1+x2,data)

ev = ld2$svd^2*(k-1)/(n-k) # 特征值
prop = ev/sum(ev) # 判别函数的贡献率
cump = round(cumsum(prop),4) # 累计贡献率
cume = round(cumsum(prop),4) # 累计贡献率
score = round(newg2$x,3) # fisher判别得分
newg2 = predict(ld2,datat)
newg2

$class
[1] 1
Levels: 1 2

$posterior
          1         2
1 0.6148464 0.3851536

$x
        LD1
1 -0.409151

因此根据fishe判别法，认为第二天会下雨。